In [12]:
using CSV, Dates, JuliaDB

In [13]:
ionex_directory = "/home/antonio/Repos/iono2/ionex_esag/"
tec_csv_directory = "/home/antonio/Repos/iono2/tec_csv_esag/"
tec_table_directory = "/home/antonio/Repos/iono2/tec_tables_esag/"

"/home/antonio/Repos/iono2/tec_tables_esag/"

In [14]:
#=
Some Files are empty and effectively missing; run the following command from the command line

 This assumes a empty_ionex_files folder

    find /home/antonio/Repos/iono2/ionex_files -type f -empty -exec mv '{}' /home/antonio/Repos/iono2/empty_ionex_files/ \;

Some files are small (Missing Data) in order to deal with this move them to the empty_ionex_files
since they have to be replace with a suitable alternative. In order to move them run the following command in the terminal

    find /home/antonio/Repos/iono2/ionex_files -type f -size -500k

Somehow some of the files are smaller thank 500k after being extracted.
run the command

find /home/antonio/Repos/iono2/ionex_files/ -type f -size -500k -exec mv '{}' /home/antonio/Repos/iono2/empty_ionex_files/ \;
=#

In [15]:
#=
Some of the files have missing data or the data is not properly formatted
it seems that files smaller than 500k have such issues.
=#

In [16]:
# OPen the file and place its contents in an array name lines
function read_file(ionex_file)
    #print("Reading", ionex_file, "\n")
    file = open(ionex_file);
    lines = readlines(file)
    close(file)
    lines  # Julia automatically returns the last statement in a function
end

read_file (generic function with 1 method)

In [17]:
function get_map_values(lines)
    # This function could be improved by reading only the TEC map portion of the file
    #print("Inside get_map_values \n")
    rms_map = 0  # Some files do not have a complete rms map 
    index = 0
    tec_values = []   # Tec values per longitud, lat(-180,180)
    tec = []          # All the tec values in a file
    dates_array = []  # Store the different time stamps
    
    for line in lines
        index += 1
        
        # Obtain time_stamp and tec values for every map (13 TEC maps in a file)
        if occursin(r"START OF TEC MAP", line)
            date_line = lines[index+1]
            date_units = []
            string = split(date_line)
            
            for x in 1:6
                push!(date_units,parse(Int,string[x]))
            end
            
            # Date units 
            year = date_units[1]
            month = date_units[2]
            day = date_units[3]
            hour = date_units[4]
            minutes = date_units[5]
            seconds = date_units[6]
            
            # Map time_stamp for a period(2 hours)
            time_stamp = DateTime(year, month, day, hour, minutes, seconds)
            push!(dates_array, time_stamp)
        end
        
        if occursin(r"START OF RMS MAP", line)
            rms_map = 1
            #print("RMS MAP reached!\n")
            return dates_array, tec
        end
        
        # Tec values per latitud in the longitud range (-180,180)
        if occursin(r"LAT/LON1/LON2/DLON/H", line) && (rms_map == 0)
            for x in 1:5
                tec_values = split(lines[index + x])
                for n in tec_values
                    #if occursin(r"^-?[0-9]*$", n)
                    if occursin(r"^-?\d*\.{0,1}\d+$", n)
                        push!(tec, n)
                    else
                        push!(tec, 0)
                        print("Issue found in ", dates_array[1], n, "\n")
                    end
                end
            end
        end
        
    end
    return dates_array, tec
end

get_map_values (generic function with 1 method)

In [18]:
# used to create a latitud, to hold 2-hour-interval Tec map data
function create_latitud(start, stop, step)
#    print("Inside create_latitud function \n")
    coordinates = []
    for i in range(start, stop=stop, step=step)
        for j in 1:73
            push!(coordinates,i)
        end
    end
    coordinates # Julia automatically returns the last statement
end
    

create_latitud (generic function with 1 method)

In [19]:
function get_coordinates()
#    print("Inside get_coordinates\n")
    latitud = create_latitud(87.5, -87.5, -2.5)
    longitud = collect(Iterators.flatten([collect(-180:5:180) for i in 1:71])) # I Still don't grok this!
    return latitud, longitud
end

get_coordinates (generic function with 1 method)

In [20]:
# Create CSV for daily tec map data.
function create_csv(directory, ionex_file)
   # print("Inside create_csv\n")
    lines = read_file(string(directory, ionex_file))
    dates_array, tec = get_map_values(lines)
    latitud, longitud = get_coordinates()
    
    csv_file = string(tec_csv_directory, ionex_file, ".csv")
    table_file = string(tec_table_directory, ionex_file, "_table")
    
    start = 1
    map_tec_values = 5183  # Total tec values in a 2 hour map period
    stop = map_tec_values
    #print("Starting to arrange data for CSV\n")
    for d in dates_array
        #print("\nStart ", start)
        time_stamp = [d for i in 1:map_tec_values]
        #print("Looking at TEC value\n")
        tec_value = [parse(Int,value) for value in tec[start:stop]]
        t = table((time_stamp=time_stamp, latitud=latitud, longitud=longitud, tec_value=tec_value); pkey = [:time_stamp, :latitud, :longitud, :tec_value])
        
        CSV.write(csv_file ,t, append=true)
        JuliaDB.save(t, table_file)
        
        start += map_tec_values
        stop += map_tec_values
    end
end


create_csv (generic function with 1 method)

## Make sure the 7306 downloaded files are uncompressed by running the command "uncompress *.Z" from within the folder containg the files
You should end up with all files ending in "i" instead of ".Z"

Some files were smaller than the average 700k, so files smaller than 500k were removed from the processing.  These are held in a TempDir folder.

In [21]:
function create_database()
    counter = 0
    directory = ionex_directory
    for f in files
        print(counter, " ", f, "\n")
        if isfile(string(tec_csv_directory, f, ".csv")) == false
            print("Writting file:", f, "\n")
            create_csv(directory, f)
        end
        counter += 1
    end
end

create_database (generic function with 1 method)

In [24]:
# Read the contents of the directory in order to process the files to create the csv 
files = readdir(ionex_directory)
@time create_database()

0 esag0010.00i
Writting file:esag0010.00i
1 esag0010.01i
Writting file:esag0010.01i
2 esag0010.02i
Writting file:esag0010.02i
3 esag0010.03i
Writting file:esag0010.03i
4 esag0010.04i
Writting file:esag0010.04i
5 esag0010.05i
Writting file:esag0010.05i
6 esag0010.06i
Writting file:esag0010.06i
7 esag0010.07i
Writting file:esag0010.07i
8 esag0010.08i
Writting file:esag0010.08i
9 esag0010.09i
Writting file:esag0010.09i
10 esag0010.10i
Writting file:esag0010.10i
11 esag0010.11i
Writting file:esag0010.11i
12 esag0010.12i
Writting file:esag0010.12i
13 esag0010.13i
Writting file:esag0010.13i
14 esag0010.14i
Writting file:esag0010.14i
15 esag0010.15i
Writting file:esag0010.15i
16 esag0010.16i
Writting file:esag0010.16i
17 esag0010.17i
Writting file:esag0010.17i
18 esag0010.18i
Writting file:esag0010.18i
19 esag0010.99i
Writting file:esag0010.99i
20 esag0020.00i
Writting file:esag0020.00i
21 esag0020.01i
Writting file:esag0020.01i
22 esag0020.02i
Writting file:esag0020.02i
23 esag0020.03i
Writt

## File jplg0430.00i from JPL has dirty data "***" showing up in strings meant to be numbers
File will be replaced by its igsg equivalent

## NOTE data will be processed per earthquake time, looking back through all the available data but just using the data to calculate the eartquake, so the data from the other months from when the earthquake happened
lets say oct 1 2000, all the years covering 20 days before oct 1 will be the rows in the observations to predict the magnitud of the eartquake.

## Loading 15GB CSV file, JuliaDB